<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/codecompletion_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
GPTNeoXColabDirName = "GPT-NeoX-Colab"
GPTNeoXColabDir = f"{workspaceDir}/{GPTNeoXColabDirName}"

# Clone CodeXGLUE Repo

In [ ]:
!git clone https://github.com/microsoft/CodeXGLUE.git &

In [ ]:
#@title Clone GPT-NeoX-Colab
%%time
%cd {workspaceDir}
# Don't use --depth 1 because that does not play nice with git-annex
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
%cd {GPTNeoXColabDir}
#%pip install --use-feature=fast-deps -q -r requirements_colab.txt
!cat requirements_colab.txt | xargs -n 1 -P 8 pip install --use-feature=fast-deps -q
%pip install --use-feature=fast-deps -q .
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.fetch_data("data/codecompletion/token_completion.tar.gz")
GPTNeoXColab.utils.colab.fetch_data("models/codecompletion.tar.gz")
#%cd {GPTNeoXColabDir}/data/codecompletion
#!tar -xzf token_completion.tar.gz
%cd {GPTNeoXColabDir}/models
!tar -xzf codecompletion.tar.gz

# Using Byte-Pair Encoding Tokenizer

In [ ]:
%cd {workspaceDir}
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json &
!wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt

In [ ]:
!pip show datasets
!pip install datasets==1.18.0
!pip install hf-transfer
!pip install lm-eval --upgrade

# HuggingFace Inference

In [ ]:
from transformers import GPTNeoXForCausalLM
import torch

%cd {workspaceDir}

# Assuming CharLevelTokenizer is properly imported and instantiated
from megatron.tokenizer.tokenizer import _GPT2BPETokenizer
tokenizer = _GPT2BPETokenizer(vocab_file=f"{workspaceDir}/gpt2-vocab.json", merge_file=f"{workspaceDir}/gpt2-merges.txt")

# Load your model
model_path = f"{GPTNeoXColabDir}models/codecompletion"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
input_text = """<s> import sys , os <EOL> import imp <EOL> from optparse import make_option <EOL> from django . conf import settings <EOL> from django . utils . importlib import import_module <EOL> from django . core . management import call_command <EOL> from django . core . management import BaseCommand <EOL> from django . db import connections <EOL> def import_app ( app_label , verbosity ) : <EOL> try : <EOL> app_path = __import__ ( app_label , { } , { } , [ app_label . split ( '<STR_LIT:.>' ) [ - <NUM_LIT:1> ] ] ) . __path__ <EOL>"""

# Tokenize and prepare input
input_ids = torch.tensor([tokenizer.tokenize(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode the generated text
generated_text = tokenizer.detokenize(output[0].tolist())

# Function to replace special tokens with original representations
def replace_special_tokens(text):
    replacements = {
        "<EOL>": "\n",  # Replace with actual newline
        "<s>": "",
        "</s>": "",     # Remove end token
        "<STR_LIT>": "STR_LITERAL",  # Example replacement, adjust as necessary
        "<NUM_LIT>": "NUM_LITERAL",   # Example replacement, adjust as necessary
    }

    for token, replacement in replacements.items():
        text = text.replace(token, replacement)

    return text.strip()  # Strip leading/trailing whitespace

# Replace special tokens in the generated text
final_text = replace_special_tokens(generated_text)

# Print the final output
print("Generated text:", final_text)


In [ ]:
!cp {workspaceDir}/gpt2-vocab.json {GPTNeoXColabDir}/models/codecompletion/vocab.json
!cp {workspaceDir}/gpt2-merges.txt {GPTNeoXColabDir}/models/codecompletion/merges.txt
!cp {GPTNeoXColabDir}/data/codecompletion/token_completion.tar.gz {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
%cd {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/dataset/py150
!tar -xzf token_completion.tar.gz

In [ ]:

%cd {workspaceDir}/CodeXGLUE/Code-Code/CodeCompletion-token/code
!python -u run_lm.py \
        --data_dir=../dataset/py150/token_completion \
        --lit_file=../dataset/py150/literals.json \
        --langs=$LANG \
        --output_dir=../dataset/py150 \
        --pretrain_dir=/content/gpt-neox/hf_model/save/location \
        --log_file=../completion_python_eval.log \
        --model_type=gpt2 \
        --block_size=2048 \
        --do_eval \
        --per_gpu_eval_batch_size=4 \
        --logging_steps=100 \
        --seed=42